In [ ]:
from flask import Flask, request, jsonify
import requests
import time

app = Flask(__name__)

API_KEY = "0c2bcbe7881e1fe6a25144bcdf33afdd"

w_cache = {}
cache_period = 3600
w_data = None

@app.route('/', methods = ['GET'])
def weather():
  city = request.args.get('city')
  error = None
  now = time.time()
  try:
    if city in w_cache and (now - w_cache[city]['period']) < cache_period:
      w_data = w_cache[city]['data']
      print("Data is cached")
      return jsonify(w_data), 203
    else:
      api_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}"
      res = requests.get(api_url)
      if res.status_code == 200:
        w_data = res.json()
        w_cache[city] = {'data' : w_data, 'period' : now}
        print(f"Cache is saved. city = {city}")
      else:
        error = "Data not found"
        return jsonify({"error": error}), 404
      res = jsonify(w_data)
      res.headers['Cache-Control'] = f'public, max-age={cache_period}'
      res.headers['Last-Modified'] = time.strftime('%a, %d %b %Y %H:%M:%S GMT', time.gmtime(now))
      print("response data from server")
      print(res.headers)
      return res
  except requests.exceptions.RequestException as err:
        print(f"Error! : {err}")
        return jsonify({"error": "Internal Server Error"}), 500

if __name__ == '__main__':
  app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Oct/2023 15:47:59] "GET /?city=Seoul HTTP/1.1" 200 -


Cache is saved. city = Seoul
response data from server
Content-Type: application/json
Content-Length: 469
Cache-Control: public, max-age=3600
Last-Modified: Sat, 14 Oct 2023 06:47:59 GMT




127.0.0.1 - - [14/Oct/2023 15:48:54] "GET /?city=Seoul HTTP/1.1" 203 -


Data is cached
